## Load Routines for Printing Matrices

In [1]:
    
include("printmat.jl")

println4Ps (generic function with 1 method)

# Bond Return after a Sudden Interest Rate Change

In [2]:
m  = 5
θ₀ = 0.02                   #initial interest rate
θ₁ = 0.03                   #interest rate after sudden change
B₀ = (1+θ₀)^(-m)
B₁ = (1+θ₁)^(-m)

ΔB = B₁ - B₀
ΔBrel = ΔB/B₀
printlnPs("Bond price before and after change of interest from $θ₀ to $θ₁: ",[B₀ B₁])

printlnPs("Exact price change: ",ΔB)
printlnPs("Exact relative price change: ",ΔBrel)

Bond price before and after change of interest from 0.02 to 0.03:      0.906     0.863
Exact price change:     -0.043
Exact relative price change:     -0.048


### From Chapter on Bonds 1

In [3]:
function BondPrice3Ps(Y,cf,mk)              #cf is a vector of all cash flows
    cfac = cf./((1+Y).^mk)                  #c/(1+y1)^t1 + c/(1+y2)^t2 + ...+ c/(1+y2)^m
    P    = sum(cfac)                        #price
    return P
end

BondPrice3Ps (generic function with 1 method)

In [4]:
cf = [zeros(4);1]                 #cash flow, 5-period zero-coupon bond          
mk = 1:5                          #time periods of the cash flows (could also do cf = 1, mk = 5)
P = BondPrice3Ps(θ₀,cf,mk)        #testing the function, using a constant interest rate of θ₀   
printlnPs("B₀ and P: ",[B₀ P])

using Roots                

ytm = fzero(y->BondPrice3Ps(y,cf,mk)-P,[-0.1;0.1])   #testing ytm calculation
printlnPs("\nθ₀ and ytm: ",[θ₀ ytm])

println("\nthe prices should be the same, the rates too")

B₀ and P:      0.906     0.906

θ₀ and ytm:      0.020     0.020

the prices should be the same, the rates too


# Macaulay's Duration

$D^M = \sum_{k=1}^{K} m_{k}\frac{cf_{k}}{\left(  1+\theta\right)  ^{m_{k}}P}$

A first-order Taylor approximation gives

$\frac{\Delta P}{P} = -D^M \times \frac{\Delta \theta}{1+\theta}$ 

In [5]:
function BondDurationMacaulay3Ps(P,cf,mk,ytm)
    cfac    = cf.*mk./((1+ytm).^(mk+1))     #c/(1+y)^2 + 2c/(1+y)^3 + ...+ Kc/(1+y)^(K+1)
    D       = sum(cfac)                     #Duration, row vector
    Da      = D/P                           #adjusted duration
    Dmac    = D*(1+ytm)/P                   #Macaulays duration
    return D,Da,Dmac
end

BondDurationMacaulay3Ps (generic function with 1 method)

In [6]:
(D,Da,Dmac) = BondDurationMacaulay3Ps(P,cf,mk,ytm)

printlnPs("Dollar duration and Macaulay's duration: ",[D Dmac])

Δθ = θ₁ - θ₀
ΔP = -D*Δθ
printlnPs("\nExact and approximate price change (according to dollar duration): ",[ΔB ΔP])

Dollar duration and Macaulay's duration:      4.440     5.000

Exact and approximate price change (according to dollar duration):     -0.043    -0.044


In [7]:
ΔPrel = -Dmac*Δθ/(1+θ₀)

printlnPs("Exact and approximate relative price change (according to Macaulay's duration): ",[ΔBrel ΔPrel])

Exact and approximate relative price change (according to Macaulay's duration):     -0.048    -0.049


In [8]:
(D,Da,Dmac) = BondDurationMacaulay3Ps(1.019,[0.04;0.04+1],1:2,0.03)
printlnPs("Different types of durations for 2-year 4% bond with ytm = 0.03: ",[D Da Dmac])

Different types of durations for 2-year 4% bond with ytm = 0.03:      1.941     1.905     1.962


# Hedging a Liability Stream

We are short one $L$ bond, which is worth $P_L$. To hedge that we buy $h$ units of bond $H$. The balance is put on a money market account $M$ to make the initial value of the portfolio zero ($V=0$)

$V=hP_{H}+M-P_{L}$

Over a short time interval, the change in the portfolio value is

$\Delta V=h\Delta P_{H}-\Delta P_{L}$, which can be approximated as

$\Delta V  =-hD_{H}^{M}P_{H}\times\frac{\Delta\theta_{H}}{1+\theta_{H}}+D_{L}^{M} P_{L}\times\frac{\Delta\theta_{L}}{1+\theta_{L}}$

In [9]:
cf = [150;250]                   #cash flow of liability stream (as positive numbers)
mk = [1;2]                       #times of cash flows   
θ₀ = 0.05                        #initial interest rate
θ₁ = 0.04                        #interest rate after sudden change
Δθ = θ₁ - θ₀

PL₀ = BondPrice3Ps(θ₀,cf,mk)      #price at initial interest rate, no face value  
PL₁ = BondPrice3Ps(θ₁,cf,mk)      #price at new interest rate

printlnPs("Value of liability at interest rates $θ₀ and $θ₁: ",[PL₀ PL₁])

(_,_,Dmac) = BondDurationMacaulay3Ps(PL₀,cf,mk,θ₀)
printlnPs("\nMacaulay's duration: ",Dmac)

Value of liability at interest rates 0.05 and 0.04:    369.615   375.370

Macaulay's duration:      1.613


### Naive Hedging with Duration of Hedge Bond DH = 0.5 (flat yield curve) 

set $h=P_{L}/P_{H}$ to get

$\frac{\Delta V}{P_{L}}\approx\left(  D_{L}^{M}-D_{H}^{M}\right)  \times
\frac{\Delta\theta}{1+\theta}$

In [10]:
PH₀ = BondPrice3Ps(θ₀,100,0.5)
PH₁ = BondPrice3Ps(θ₁,100,0.5)      

h = PL₀/PH₀

printlnPs("A zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h: ",[PH₀ h])

ΔV_exact = h*(PH₁-PH₀) - (PL₁-PL₀)
printlnPs("\n(relative) % change of hedge portfolio value (exact): ",100*ΔV_exact/PL₀)

ΔVrel_approx = (Dmac - 0.5)*Δθ/(1+θ₀)
printlnPs("\n(relative) % change of hedge portfolio value (approx): ",100*ΔVrel_approx)

A zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h:     97.590     3.787

(relative) % change of hedge portfolio value (exact):     -1.077

(relative) % change of hedge portfolio value (approx):     -1.060


### Duration Hedging  (flat yield curve)

instead, set 

$h =\frac{D_{L}^{M}}{D_{H}^{M}}\times\frac{P_{L}}{P_{H}}$ to get

$\frac{\Delta V}{P_{L}}\approx D_{L}^{M}\times\left(  \frac{\Delta\theta_{L} }{1+\theta_{L}}-\frac{\Delta\theta_{H}}{1+\theta_{H}}\right)$

In [11]:
PH₀ = BondPrice3Ps(θ₀,100,Dmac)             
PH₁ = BondPrice3Ps(θ₁,100,Dmac)      
h = Dmac/Dmac * PL₀/PH₀
printlnPs("A zero coupon bond with m=$(round(Dmac,2)) and face value of 100: price and hedge ratio h: ",[PH₀ h])

ΔV_exact = h*(PH₁-PH₀) - (PL₁-PL₀)
printlnPs("\n(relative) % change of hedge portolio value (exact): ",100*ΔV_exact/PL₀)

ΔVrel_approx = Dmac*(Δθ/(1+θ₀) - Δθ/(1+θ₀)) 
printlnPs("\n(relative) % change of hedge portolio value (approx): ",100*ΔVrel_approx)

A zero coupon bond with m=1.61 and face value of 100: price and hedge ratio h:     92.430     3.999

(relative) % change of hedge portolio value (exact):     -0.001

(relative) % change of hedge portolio value (approx):      0.000


# Hedging a Liability II (Example from Figure in the Lecture Notes)

In [12]:
cf = ones(10)*0.2               #cash flow of liability stream (as positive numbers)
mk = 1:10                       #times of cash flows   
θ₀ = 0.05                       #initial interest rate
θ₁ = 0.03                       #interest rate after sudden change
Δθ = θ₁ - θ₀

PL₀ = BondPrice3Ps(θ₀,cf,mk)       #price at initial interest rate
PL₁ = BondPrice3Ps(θ₁,cf,mk)       #price at new interest rate
printlnPs("Value of liability at interest rates $θ₀ and $θ₁: ",[PL₀ PL₁])

(_,_,Dmac) = BondDurationMacaulay3Ps(PL₀,cf,mk,θ₀)
printlnPs("Duration: ",Dmac)

PH₀ = BondPrice3Ps(θ₀,1,3)          #hedge bond, paying 1 after 3 years
PH₁ = BondPrice3Ps(θ₁,1,3)      
h = Dmac/3 * PL₀/PH₀
printlnPs("\nValue of hedge bond with m=3 at interest rates $θ₀ and $θ₁: ",[PH₀ PH₁])
printlnPs("The hedge ratio (h) is $(round(h,3)) and the relative invested amount is $(round(h*PH₀/PL₀,3))")

Value of liability at interest rates 0.05 and 0.03:      1.544     1.706
Duration:      5.099

Value of hedge bond with m=3 at interest rates 0.05 and 0.03:      0.864     0.915
The hedge ratio (h) is 3.039 and the relative invested amount is 1.7
